In [ ]:
!pip install requests

In [ ]:
!pip install pyodbc pandas

In [ ]:
# Declare Variables

tenant_id = 'tenant-id'
client_id = 'client_id'
client_secret = 'client_secret'
redirect_uri = 'http://localhost'  # This should match the redirect URI set in Azure AD
scope = 'https://api.fabric.microsoft.com/Workspace.ReadWrite.All'
authorization_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/authorize'
token_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
authentication = 'ActiveDirectoryServicePrincipal'



In [ ]:
import requests
import json
import csv
from urllib.parse import urlencode, urlparse, parse_qs

# Azure AD and API endpoint details

# Step 1: Get user authorization
def get_authorization_code():
    params = {
        'client_id': client_id,
        'response_type': 'code',
        'redirect_uri': redirect_uri,
        'response_mode': 'query',
        'scope': scope
    }
    auth_request_url = f'{authorization_url}?{urlencode(params)}'
    print(f'Please go to the following URL and authorize the application: {auth_request_url}')

    redirect_response = input('Paste the full redirect URL here: ')
    parsed_url = urlparse(redirect_response)
    authorization_code = parse_qs(parsed_url.query)['code'][0]
    return authorization_code

# Step 2: Exchange authorization code for access token
def get_tokens(authorization_code):
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    body = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'client_secret': client_secret,
        'code': authorization_code,
        'redirect_uri': redirect_uri,
        'scope': scope
    }
    response = requests.post(token_url, headers=headers, data=body)
    response.raise_for_status()
    return response.json()

# Function to call Fabric API for Lakehouses
def call_lakehouse_api(workspace_id, access_token):
    
    api_url = f'https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/lakehouses'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(api_url, headers=headers)
    response.raise_for_status()
    return response.json()

# Function to call Fabric API for Warehouses
def call_warehouse_api(workspace_id, access_token):
    api_url = f'https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/warehouses'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(api_url, headers=headers)
    response.raise_for_status()
    return response.json()

# Function to save data to CSV
def save_to_csv(data, filename='output.csv'):
    keys = data[0].keys()
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

def main():
    try:
        # Get authorization code from user
        authorization_code = get_authorization_code()
        
        # Exchange the authorization code for access and refresh tokens
        token_response = get_tokens(authorization_code)
        
        # Extract the access token from the response
        access_token = token_response['access_token']
        
        # List of workspace IDs (for example purposes)
        workspace_ids = ['Workspaceid-1','Workspaceid-2','Workspaceid-3']
        
        all_data = []

        for workspace_id in workspace_ids:
            lakehouse_response = call_lakehouse_api(workspace_id, access_token)
            warehouse_response = call_warehouse_api(workspace_id, access_token)
            
            for lakehouse in lakehouse_response['value']:
                lakehouse_data = {
                    'id': lakehouse['id'],
                    'type': lakehouse['type'],
                    'displayName': lakehouse['displayName'],
                    'description': lakehouse['description'],
                    'workspaceId': lakehouse['workspaceId'],
                    'oneLakeTablesPath': lakehouse['properties']['oneLakeTablesPath'],
                    'oneLakeFilesPath': lakehouse['properties']['oneLakeFilesPath'],
                    'sqlEndpointConnectionString': lakehouse['properties']['sqlEndpointProperties']['connectionString'],
                    'provisioningStatus': lakehouse['properties']['sqlEndpointProperties']['provisioningStatus']
                }
                all_data.append(lakehouse_data)
            for warehouse in warehouse_response['value']:
                warehouse_data = {
                    'id': warehouse['id'],
                    'type': warehouse['type'],
                    'displayName': warehouse['displayName'],
                    'description': warehouse['description'],
                    'workspaceId': warehouse['workspaceId'],
                    'oneLakeTablesPath':'NA',
                    'oneLakeFilesPath': 'NA',
                    'sqlEndpointConnectionString': warehouse['properties']['connectionInfo']
                }
                all_data.append(warehouse_data)
        
        # Save all collected data to a CSV file
        save_to_csv(all_data)
        print('All information is saved in a CSV File')
        
    except requests.exceptions.HTTPError as err:
        print(f'HTTP error occurred: {err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

if __name__ == '__main__':
    main()


In [ ]:
import requests
import pyodbc
import pandas as pd
import csv

def read_connection_details(csv_file):
    connections = []
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['type'] in ['Lakehouse', 'Warehouse']:
                connection_string = row['sqlEndpointConnectionString']
                connections.append({
                    'connection_string': connection_string,
                    'database_name': row['displayName']
                })
    return connections

def execute_query_and_save_to_csv(connection_string, database_name, query, output_file):
    try:
        conn_str = (
            f"Driver={{ODBC Driver 18 for SQL Server}};"
            f"Server={connection_string};"
            f"Database={database_name};"
            f"Authentication={authentication};"
            f"Uid={client_id};"
            f"Pwd={client_secret};"
            f"Authority Id={tenant_id};"
            f"Encrypt=yes;"
            f"TrustServerCertificate=no;"
            f"Connection Timeout=30;"
        )
        conn = pyodbc.connect(conn_str)
        df = pd.read_sql_query(query, conn)
        df.to_csv(output_file, index=False)
        print(f'Results saved to {output_file}')
    except Exception as e:
        print(f'Error: {e}')
    finally:
        if 'conn' in locals():
            conn.close()

def main():

    csv_file = 'output.csv'
    query = "SELECT * FROM [queryinsights].[exec_requests_history]"
    
    connections = read_connection_details(csv_file)

    for i, conn_details in enumerate(connections):
        
        output_file = f'query_results_{i+1}.csv'
        print(conn_details['connection_string'])
        print(conn_details['database_name'])
        print(output_file)
        execute_query_and_save_to_csv(
            conn_details['connection_string'],
            conn_details['database_name'],
            query,
            output_file
        )

if __name__ == '__main__':
    main()
